In [2]:
import rasterio
import xvec
import pandas as pd

In [11]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]

In [ ]:
def compute_sightlines_mnt_elevations():
    
    #=================
    # ITERATE over SIGHTLINES
    #=================
    POINT_TYPE_START = -1 
    POINT_TYPE_END = -2

    tiles_values = []
    #sight_point_z_values = []

    for uid, start_point, end_point, sight_points in sightlines.itertuples():
        # start point
        point_type = POINT_TYPE_START
        tile_x,tile_y,xcoord,ycoord=mnt_xy_to_tile(*start_point)
        tiles_values.append([uid,point_type,(tile_x,tile_y),xcoord,ycoord])

        # end point
        point_type = POINT_TYPE_END
        tile_x,tile_y,xcoord,ycoord=mnt_xy_to_tile(*end_point)
        tiles_values.append([uid,point_type,(tile_x,tile_y),xcoord,ycoord])

        # sight points    
        for pt,pt_index in zip(sight_points,range(len(sight_points))):
            tile_x,tile_y,xcoord,ycoord=mnt_xy_to_tile(*pt)
            tiles_values.append([uid,pt_index,(tile_x,tile_y),xcoord,ycoord])

    df_points_tiles = pd.DataFrame(tiles_values,
                                  columns=['uid','point_id','tile','xcoord_in_tile','ycoord_in_tile']).set_index('uid')
    
    #=================
    # ASSIGN MNT elevation after grouping each point by tile reference
    #=================
    df_points_tiles_group = df_points_tiles.groupby('tile')
    z_values = []

    for tile, group in df_points_tiles_group:
        tile_x,tile_y = tile
        # open tile
        tile_x_str = f'{tile_x:04}'
        tile_y_str = f'{tile_y:04}'


        tile_section = gdf_tiles[(gdf_tiles.tile_x==tile_x_str) & (gdf_tiles.tile_y==tile_y_str)]
        if len(tile_section)>0:
            tile_section = tile_section.iloc[0].tile_section        
            tile_filename = os.path.join(MNT_TARGET_PATH,'ASC')
            tile_filename = os.path.join(tile_filename,tile_section)
            tile_filename = os.path.join(tile_filename,f"RGEALTI_FXX_{tile_x_str}_{tile_y_str}_MNT_LAMB93_IGN69.asc.bz2")

            z_grid = pickle.load(bz2.open(tile_filename,  'rb' ))
            #display(tile_filename)    
            for uid,point_id,tt,xcoord_in_tile,ycoord_in_tile in group.itertuples():
                z_values.append([uid,point_id,z_grid[ycoord_in_tile,xcoord_in_tile],True])
        else:
            for uid,point_id,tt,xcoord_in_tile,ycoord_in_tile in group.itertuples():
                z_values.append([uid,point_id,MNT_NO_DATA_VALUE,False])

    df_z = pd.DataFrame(z_values,
                        columns=['uid','point_id','z','valid'])    
    df_z = df_z.sort_values(by=['uid','point_id'])
    df_z = df_z.set_index('uid')
    
    #join results to sight points
    sightlines=sightlines.join(df_z[df_z.point_id==POINT_TYPE_START][['z']].rename(columns={'z':'z_start'}))
    sightlines=sightlines.join(df_z[df_z.point_id==POINT_TYPE_END][['z']].rename(columns={'z':'z_end'}))
    sightlines=sightlines.join(df_z[df_z.point_id>=0][['z']].groupby('uid').aggregate(list).rename(columns={'z':'z_sight_points'}))    
    
    #=================
    # EXPORT SIGNLINE MNT elevation to pickle 
    #=================
    # fs_cache.save_to_pickle(sightlines,'road_sightlines',f'{PARAM_insee_layer_name}_{zone_code}_sightlines_mnt_dataframe')
    
    #=================
    # Compute all road slope indicators
    #=================
    slope_values = []
    #sight_point_z_values = []

    for uid, road_row in sightlines.iterrows():
        slope_degree,slope_percent,n_slopes,slope_is_valid = compute_slope(**road_row)
        slope_values.append([uid,
                             slope_degree,
                             slope_percent,
                             n_slopes,
                             slope_is_valid])

    df_slopes = pd.DataFrame(slope_values,
                             columns=['uid',
                                      'slope_degree', 
                                      'slope_percent',
                                      'n_slopes',
                                      'slope_is_valid']).set_index('uid')
    #=================
    # EXPORT ROADS MNT indicators as pickle
    #=================
    
    # fs_cache.save_to_pickle(df_slopes,
    #                         'road_indicators',
    #                         f'{PARAM_insee_layer_name}_{zone_code}_road_mnt_indicators',
    #                         verbose=display)